# London Boroughs

## 1. Import data of London boroughs, post codes and geo information

In [53]:
import requests
import numpy as np

In [20]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f9ba163b411649f49b1093342697666e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='WbgQJlT2ZIz1pR_6pF3gvMK-FT62wWkO8CDRLmcUUKvA',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_f9ba163b411649f49b1093342697666e.get_object(Bucket='capstoneproject-donotdelete-pr-qewn6395ztg9eo',Key='ukpostcodes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,id,postcode,latitude,longitude
0,1,AB10 1XG,57.144165,-2.114848
1,2,AB10 6RN,57.137880,-2.121487
2,3,AB10 7JB,57.124274,-2.127190
3,4,AB11 5QN,57.142701,-2.093295
4,5,AB11 6UL,57.137547,-2.112233


In [21]:

body = client_f9ba163b411649f49b1093342697666e.get_object(Bucket='capstoneproject-donotdelete-pr-qewn6395ztg9eo',Key='LondonBorough.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
df_data_0.head()


,postcode,Borough
0,E1,"Whitechapel, Stepney, Mile End"
1,E1W,Wapping
2,E2,"Bethnal Green, Shoreditch"
3,E3,"Bow, Bromley-by-Bow"
4,E4,"Chingford, Highams Park"


## 2. Combine these 2 data frames into 1 

In [22]:
# split postcode front and end parts in order to merge 2 dataframes
df_data_1['A'], df_data_1['B'] = df_data_1['postcode'].str.split(' ', 1).str
df_data_1.head()

,id,postcode,latitude,longitude,A,B
0,1,AB10 1XG,57.144165,-2.114848,AB10,1XG
1,2,AB10 6RN,57.137880,-2.121487,AB10,6RN
2,3,AB10 7JB,57.124274,-2.127190,AB10,7JB
3,4,AB11 5QN,57.142701,-2.093295,AB11,5QN
4,5,AB11 6UL,57.137547,-2.112233,AB11,6UL


In [23]:
#left join
df = pd.merge(left=df_data_0, right=df_data_1 , left_on='postcode', right_on='A')
df.head()

,postcode_x,Borough,id,postcode_y,latitude,longitude,A,B
0,N1,"Islington, Barnsbury, Canonbury",772525,N1 9QQ,51.532829,-0.114887,N1,9QQ
1,N1,"Islington, Barnsbury, Canonbury",772526,N1 9QR,51.534427,-0.112528,N1,9QR
2,N1,"Islington, Barnsbury, Canonbury",772527,N1 9QS,51.538645,-0.103543,N1,9QS
3,N1,"Islington, Barnsbury, Canonbury",772528,N1 9QT,51.532677,-0.112082,N1,9QT
4,N1,"Islington, Barnsbury, Canonbury",772529,N1 9QU,51.534365,-0.115386,N1,9QU


## 3. Combine areas in one postal area, by means of grouping by postcode front part, e.g. N1, keeping only one post code, and therefore one latitude & longitude

In [24]:
# clean the data frame
df1 = df.drop_duplicates(subset='A', keep="first")
df1.reset_index(inplace = True)
df1.rename(columns = {'postcode_y':'postcode'}, inplace = True)
df1.drop('postcode_x', axis = 1, inplace = True)
df1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,index,Borough,id,postcode,latitude,longitude,A,B
0,0,"Islington, Barnsbury, Canonbury",772525,N1 9QQ,51.532829,-0.114887,N1,9QQ
1,2149,East Finchley,764225,N2 9ED,51.590841,-0.164371,N2,9ED
2,2666,Finchley Central,761725,N3 3DY,51.589698,-0.202630,N3,3DY
3,3274,"Finsbury Park, Manor House",760725,N4 4QJ,51.575607,-0.107157,N4,4QJ
4,4016,Highbury,760425,N5 2LT,51.561173,-0.097080,N5,2LT


## 4. Print the number of rows

In [25]:
df1.shape

(93, 8)

### Woohoo, that's the end. I have now obtained geographical coordinates of London Boroughs

## Oh, not yet.

## 5. Generate map

In [26]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [27]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


altair-3.2.0         | 770 KB    | #################################

In [28]:
address = 'London' #'New York City, NY'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [31]:
# create map of Toronto using latitude and longitude values
map_London = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df1['latitude'], df1['longitude'], df1['Borough']):
    label = ' {}'.format( borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_London)  
    
map_London

### Make sure the map can be displayed in github

In [142]:
from IPython.core.display import HTML

HTML(map_London._repr_html_())

## 6. Clustering

In [32]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

### Define Foursquare Credentials and Version

In [33]:
# Foursquare credential
CLIENT_ID = '4QVLUBH3IHBL1YISHGZZVBMAIY1NMCE12MYGGASQSXXY1E1P' # your Foursquare ID
CLIENT_SECRET = 'CHXJGKOW45ELNUQFQGVLKDZRFSN3OQTS05TM0A3BUBLVHNUT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [ ]:
# LIMIT = 100 # limit of number of venues returned by Foursquare API
# radius = 500 # define radius

### Now write the code to run the function below on each neighborhood and create a new dataframe called *London_venues*.

In [34]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT = 100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode',
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
London_venues = getNearbyVenues(names=df1['postcode'],
                                   latitudes=df1['latitude'],
                                   longitudes=df1['longitude']
                                  )

N1 9QQ
N2 9ED
N3 3DY
N4 4QJ
N5 2LT
N6 6EL
N7 9SZ
N8 9AX
N9 9PJ
N10 3XB
N11 3GS
N12 9JR
N13 6AR
N14 7NA
N15 6BJ
N16 9PS
N17 9LP
N18 2TZ
N19 5EG
N20 9EY
N21 3JQ
N22 8XY
NW1 9RL
NW2 7PH
NW3 7UA
NW4 4HP
NW5 4EG
NW6 7SR
NW7 4AS
NW8 9JD
NW9 9RR
NW10 9TG
NW11 9UL
SE1 9BL
SE2 9PH
SE3 9XA
SE4 2HU
SE5 9NX
SE6 4PH
SE7 8TP
SE8 5BL
SE9 6UE
SE10 9UQ
SE11 6AL
SE12 9NE
SE13 7ET
SE14 6RU
SE15 6PN
SE16 6YR
SE17 3TF
SE18 7QT
SE19 3JR
SE20 8TJ
SE21 8EA
SE22 9AA
SE23 3XT
SE24 0XT
SE25 6UG
SE26 6QW
SE27 9HG
SE28 8ES
SW2 9GQ
SW3 6NA
SW4 9QH
SW5 9TT
SW6 7TP
SW7 5RN
SW8 4UN
SW9 9PQ
SW10 9LQ
SW11 6DQ
SW12 9QT
SW13 9NQ
SW14 8QN
SW15 6HQ
SW16 6RW
SW17 9PE
SW18 5RT
SW19 8NJ
SW20 9LE
W2 7JU
W3 8UZ
W4 5LZ
W5 9EF
W6 9JQ
W7 3RQ
W8 7XY
W9 3NW
W10 6LH
W11 4JA
W12 9UN
W13 9UD
W14 9RS


### Analyse borough and  group rows by postcode and by taking the mean of the frequency of occurrence of each category

In [47]:
# one hot encoding
London_onehot = pd.get_dummies(London_venues[['Venue Category']], prefix="", prefix_sep="")

# add postcode column back to dataframe
London_onehot['Postcode'] = London_venues['Postcode'] 

# move postcode column to the first column
fixed_columns = [London_onehot.columns[-1]] + list(London_onehot.columns[:-1])
London_onehot = London_onehot[fixed_columns]

London_onehot.head()

,Postcode,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Volleyball Court,Warehouse Store,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,N1 9QQ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,N1 9QQ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,N1 9QQ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,N1 9QQ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,N1 9QQ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
London_grouped = London_onehot.groupby('Postcode').mean().reset_index()

In [49]:
London_grouped.head()

,Postcode,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Volleyball Court,Warehouse Store,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,N1 9QQ,0.015625,0.000000,0.0,0.0,0.0,0.0,0.015625,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015625,0.0
1,N10 3XB,0.000000,0.020408,0.0,0.0,0.0,0.0,0.000000,0.0,0.020408,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,N11 3GS,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,N12 9JR,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,N13 6AR,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


### Find the top 5 most common venues of each postal area, and store them in a dataframe 

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Postcode'] = London_grouped['Postcode']
for ind in np.arange(London_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,N1 9QQ,Café,Pub,Vegetarian / Vegan Restaurant,Thai Restaurant,Hotel
1,N10 3XB,Café,Pub,Coffee Shop,Pizza Place,Restaurant
2,N11 3GS,Sporting Goods Shop,Gym / Fitness Center,Residential Building (Apartment / Condo),Café,Zoo
3,N12 9JR,Tennis Court,Auto Garage,Rental Car Location,Turkish Restaurant,Zoo
4,N13 6AR,Food & Drink Shop,Bus Stop,Greek Restaurant,Grocery Store,Rental Car Location


In [55]:
venues_sorted.shape

(93, 6)

### Run *k*-means to cluster the postal area into 5 clusters.

In [62]:
# set number of clusters
kclusters = 5

London_grouped_clustering = London_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(London_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 4, 0, 0, 0, 0], dtype=int32)

In [64]:
#uncomment if exists a column named 'ClusterLabels'
venues_sorted.drop(['ClusterLabels'], axis = 1, inplace = True) 

In [65]:
# add clustering labels
venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

London_merged = df1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
London_merged = London_merged.join(venues_sorted.set_index('Postcode'), on='postcode')


In [66]:
London_merged.head() # check the last columns!

,index,Borough,id,postcode,latitude,longitude,A,B,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0,"Islington, Barnsbury, Canonbury",772525,N1 9QQ,51.532829,-0.114887,N1,9QQ,0,Café,Pub,Vegetarian / Vegan Restaurant,Thai Restaurant,Hotel
1,2149,East Finchley,764225,N2 9ED,51.590841,-0.164371,N2,9ED,2,Coffee Shop,Café,Platform,Pub,Grocery Store
2,2666,Finchley Central,761725,N3 3DY,51.589698,-0.202630,N3,3DY,0,Sandwich Place,Bus Stop,Bakery,Grocery Store,Middle Eastern Restaurant
3,3274,"Finsbury Park, Manor House",760725,N4 4QJ,51.575607,-0.107157,N4,4QJ,0,Nature Preserve,Tapas Restaurant,Convenience Store,Train Station,Bus Stop
4,4016,Highbury,760425,N5 2LT,51.561173,-0.097080,N5,2LT,0,Café,Pizza Place,Pub,Gym / Fitness Center,Sports Bar


In [67]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(London_merged['latitude'], London_merged['longitude'], London_merged['Borough'], London_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [143]:
HTML(map_clusters._repr_html_())

In [80]:
London_merged[London_merged.Borough.str.contains("Kensington")]

,index,Borough,id,postcode,latitude,longitude,A,B,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
66,50077,South Kensington,237525,SW7 5RN,51.494962,-0.181018,SW7,5RN,0,Hotel,Exhibit,Italian Restaurant,Mediterranean Restaurant,Burger Joint
86,66855,Kensington,98625,W8 7XY,51.498288,-0.198319,W8,7XY,0,Café,Italian Restaurant,Pub,Supermarket,Persian Restaurant
88,68060,"Ladbroke Grove, North Kensington",117925,W10 6LH,51.519967,-0.219015,W10,6LH,0,Gym / Fitness Center,Park,Breakfast Spot,Theater,Indoor Play Area
92,70626,West Kensington,114825,W14 9RS,51.484964,-0.210261,W14,9RS,0,Pub,Park,Gym / Fitness Center,Café,Tennis Court
